# Store

> Module for constructing AppletStore.

In [ ]:
#| default_exp store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import yaml
import json
import os

from llmcam.oas_to_requests import toolbox_schema
from typing import Optional

In [ ]:
#| export
ToolBox = {}

## Dynamic API installation

In [ ]:
#| export
def load_oas(
    oas_url: str = "https://tie.digitraffic.fi/swagger/openapi.json",  # OpenAPI Specification URL
    destination: str = "api/road_digitraffic.json",  # Destination file
    overwrite: bool = False  # Overwrite existing file
) -> dict:  # OpenAPI Specification
    """Load OpenAPI Specification from URL or file."""
    # Create destination directory if it does not exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)

    # Download OpenAPI Specification if it does not exist or overwrite is True
    if not os.path.exists(destination) or overwrite:
        r = requests.get(oas_url)
        with open(destination, "w") as f:
            f.write(r.text)

    # Load OpenAPI Specification
    with open(destination, "r") as f:
        if destination.endswith(".json"):
            return json.load(f)
        elif destination.endswith(".yaml") or destination.endswith(".yml"):
            return yaml.load(f)
        else:
            raise ValueError("Invalid file format")

In [ ]:
show_doc(load_oas)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/store.py#L22){target="_blank" style="float:right; font-size:smaller"}

### load_oas

>      load_oas (oas_url:str='https://tie.digitraffic.fi/swagger/openapi.json',
>                destination:str='api/road_digitraffic.json',
>                overwrite:bool=False)

*Load OpenAPI Specification from URL or file.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| oas_url | str | https://tie.digitraffic.fi/swagger/openapi.json | OpenAPI Specification URL |
| destination | str | api/road_digitraffic.json | Destination file |
| overwrite | bool | False | Overwrite existing file |
| **Returns** | **dict** |  | **OpenAPI Specification** |

Usage to download three DigiTraffic endpoints

In [ ]:
#| export
road_digitraffic = load_oas(
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    destination="api/road_digitraffic.json",
    overwrite=False
)
train_digitraffic = load_oas(
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    destination="api/train_digitraffic.json",
    overwrite=False
)
marine_digitraffic = load_oas(
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    destination="api/marine_digitraffic.json",
    overwrite=False
)

Test functions with GPT:

In [ ]:
import textwrap
from llmcam.fn_to_fc import complete, form_msg
from llmcam.oas_to_requests import generate_request

In [ ]:
#| eval: false
messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg("user", "Get me information about the trains departing on 2017-11-09 with train number 1.")
]
complete(messages, toolbox_schema("https://rata.digitraffic.fi", train_digitraffic), aux_fn=generate_request)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get me information about the trains departing on 2017-11-09 with train number 1.
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_WyALfBABJY1UNmWajdbgOutw', 'function': {'arguments': '{"departure_date":"2017-11-09","train_number":1}', 'name': 'getTrainByTrainNumberAndDepartureDate'}, 'type': 'function'}]}
>> Tool:
{"departure_date": "2017-11-09", "train_number": 1, "getTrainByTrainNumberAndDepartureDate":
[{"trainNumber": 1, "departureDate": "2017-11-09", "operatorUICCode": 10, "operatorShortCode": "vr",
"trainType": "IC", "trainCategory": "Long-distance", "commuterLineID": "", "runningCurrently":
false, "cancelled": false, "version": 231593807888, "timetableType": "REGULAR",
"timetableAcceptanceDate": "2017-07-21T11:27:05.000Z", "timeTableRows": [{"stationShortCode": "HKI",
"stationUICCode": 1, "countryCode": "FI", "type": "DEPARTURE", "

In [ ]:
#| eval: false
messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg("user", "Are there any active nautical warnings?")
]
complete(messages, toolbox_schema("https://meri.digitraffic.fi", marine_digitraffic), aux_fn=generate_request)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Are there any active nautical warnings?
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_S3mAaovBQjkeNDjSlsbbk95q', 'function': {'arguments': '{"url":"https://meri.digitraffic.fi/api/nautical-warning/v1/warnings/active","method":"get"}', 'name': 'get_api_nautical_warning_v1_warnings_active'}, 'type': 'function'}]}
>> Tool:
{"url": "https://meri.digitraffic.fi/api/nautical-warning/v1/warnings/active", "method": "get",
"get_api_nautical_warning_v1_warnings_active": {"type": "FeatureCollection", "features": [{"type":
"Feature", "geometry": {"type": "Polygon", "coordinates": [[[23.35803893344, 64.1366363392165],
[23.938027213478897, 64.4119896036909], [24.1078447347889, 64.58443222591849], [23.9197195479886,
64.7294449566291], [23.6817198966139, 64.8013598139946], [23.357787405160398, 64.8191679117966],
[22.9866304792208, 64.72628481643409], 

In [ ]:
#| export
def add_api_tools(
    service_name: str,  # Name of the API service
    base_url: str,  # Base URL of the API service
    oas_url: Optional[str] = None,  # OpenAPI Specification URL
    oas_destination: Optional[str] = None # OpenAPI Specification destination file
):
    """Add API tools to the toolbox."""
    # Load OpenAPI Specification
    if oas_url is None:
        oas_url = f"{base_url}/swagger/openapi.json"
    if oas_destination is None:
        oas_destination = f"api/{service_name}.json"
    oas = load_oas(oas_url, oas_destination, overwrite=True)

    # Create tool schema and append to toolbox
    global ToolBox
    ToolBox[service_name] = toolbox_schema(base_url, oas)

In [ ]:
show_doc(add_api_tools)

---

### add_api_tools

>      add_api_tools (base_url:str, oas_url:Optional[str]=None,
>                     oas_destination:Optional[str]='api/temp.json')

*Add API tools to the toolbox.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| base_url | str |  | Base URL of the API service |
| oas_url | Optional | None | OpenAPI Specification URL |
| oas_destination | Optional | api/temp.json | OpenAPI Specification destination file |

## Dynamic functions from installed libraries

It is possible to dynamically import a function in installed library and execute it with `importlib`.

In [ ]:
#| export
from importlib import import_module
from typing import Callable, Any
from llmcam.fn_to_fc import tool_schema

In [ ]:
import json

In [ ]:
func: Callable = getattr(import_module("llmcam.fn_to_fc"), "ask_gpt4v_about_image_file", None)
func

<function llmcam.fn_to_fc.ask_gpt4v_about_image_file(path: str) -> str>

In [ ]:
#| eval: false
json.loads(func("data/cap_2024.09.28_15:59:06_Presidentinlinna.jpg"))

{'timestamp': '2024-09-28T15:59:06',
 'location': 'Presidentinlinna',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 7,
  'building_height_range': '3-5 stories'},
 'vehicles': {'number_of_vehicles': 25,
  'number_of_available_parking_space': 5},
 'waterbodies': {'visible': True, 'type': 'harbor', 'number_of_boats': 10},
 'street_lights': {'number_of_street_lights': 10},
 'people': {'approximate_number': 30},
 'lighting': {'time_of_day': 'afternoon',
  'artificial_lighting': 'not prominent'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'daylight'}}

Auxiliary function to dynamically import and execute functions as defined in tools:

In [ ]:
#| export
def execute_function(
    function_name: str,  # Name of the function
    tools: list = [],  # The toolbox schema
    **kwargs  # Keyword arguments
) -> Any:  # Function output
    """Execute function with specified name."""
    # Get function from toolbox
    for tool in tools:
        if tool["function"]["name"] == function_name:
            module = tool["function"]["parameters"]["properties"]["module"]["default"]

    # Import module and get function
    if module is None:
        raise ValueError("Module not found")
    if module == "builtins":
        func: Callable = getattr(__builtins__, function_name, None)
    else:
        func: Callable = getattr(import_module(module), function_name, None)
    
    # Execute function
    if func is None:
        raise ValueError("Function not found")
    return func(**kwargs)

Usage with existing function:

In [ ]:
#| eval: false
json.loads(execute_function(
    "ask_gpt4v_about_image_file", 
    tools=[tool_schema(func)], 
    path="data/cap_2024.09.28_15:59:06_Presidentinlinna.jpg"
))

{'timestamp': '2024-09-28T15:59:06',
 'location': 'unknown',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 10,
  'building_height_range': '3-5 stories'},
 'vehicles': {'number_of_vehicles': 20,
  'types': ['cars', 'vans'],
  'number_of_available_parking_space': 5},
 'waterbodies': {'visible': True, 'type': 'harbor', 'number_of_boats': 10},
 'street_lights': {'number_of_street_lights': 15},
 'people': {'approximate_number': 30},
 'lighting': {'time_of_day': 'afternoon',
  'artificial_lighting': 'not prominent'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'daylight'}}

Utility to add functions from installed libraries to ToolBox.

In [ ]:
#| export
def add_function_tools(
    service_name: str,  # Name of the service
    function_names: list[str],  # List of function names (with module prefix)
):
    """Add function tools to the toolbox."""
    # Initialize tools
    tools = []

    # Import functions
    for function_name in function_names:
        # Get module prefix
        module_prefix = function_name.split(".")
        if len(module_prefix) == 1:
            module_prefix = "builtins"
        else:
            module_prefix = ".".join(module_prefix[:-1])

        # Get function name without module prefix
        func_name = function_name.split(".")[-1]

        # Import function
        if module_prefix == "builtins":
            func: Callable = getattr(__builtins__, func_name, None)
        else:
            func: Callable = getattr(import_module(module_prefix), func_name, None)

        # Raise error if function not found
        if func is None:
            raise ValueError(f"Function not found: {function_name}")
        
        # Create tool schema
        tools.append(tool_schema(func))

    # Append tools to toolbox
    global ToolBox
    for function_name in function_names:
        ToolBox[service_name] = tools

In [ ]:
show_doc(add_function_tools)

---

### add_function_tools

>      add_function_tools (service_name:str, function_names:list[str])

*Add function tools to the toolbox.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| service_name | str | Name of the service |
| function_names | list | List of function names (with module prefix) |

## ToolBox management

From the previous sections, we have implemented the utilities to add tools from API and functions. In this section, we can implement more utilities to manage the ToolBox, including:  

- Remove or Clean toolbox
- Save and load toolbox from `.json` file
- Select services and form usable tools 

In [ ]:
#| export
def remove_service(service_name: str):
    """Remove service from toolbox."""
    global ToolBox
    if service_name in ToolBox:
        del ToolBox[service_name]

#| export
def clean_toolbox():
    """Remove all services from toolbox."""
    global ToolBox
    ToolBox = {}

In [ ]:
#| export
def save_toolbox(destination: str = "toolbox.json"):
    """Save toolbox to file."""
    with open(destination, "w") as f:
        json.dump(ToolBox, f)
    
#| export
def load_toolbox(destination: str = "toolbox.json"):
    """Load toolbox from file."""
    global ToolBox
    with open(destination, "r") as f:
        ToolBox = json.load(f)

In [ ]:
#| export
def extract_tools_from_services(
    services: list[str] = []  # List of service names
) -> list:  # List of tools
    """Extract tools from services."""
    # Initialize tools
    tools = []

    # Extract tools from services
    global ToolBox
    for service in services:
        if service in ToolBox:
            tools.extend(ToolBox[service])

    # Raise error if no tools found
    if len(tools) == 0:
        raise ValueError("No tools found")
    
    # Raise error if too many tools
    if len(tools) > 128:
        raise ValueError("Too many tools for using GPT-4. Maximum number of tools is 128.")

    # Return tools
    return tools

In [ ]:
show_doc(extract_tools_from_services)

---

### extract_tools_from_services

>      extract_tools_from_services (services:list[str]=[])

*Extract tools from services.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| services | list | [] | List of service names |
| **Returns** | **list** |  | **List of tools** |

For our demo, we can add some default services from DigiTraffic and our existing `llmcam` module:

In [ ]:
#| export
add_api_tools(
    service_name="road_digitraffic",
    base_url="https://tie.digitraffic.fi",
    oas_url="https://tie.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/road_digitraffic.json"
)

In [ ]:
#| export
add_api_tools(
    service_name="train_digitraffic",
    base_url="https://rata.digitraffic.fi",
    oas_url="https://rata.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/train_digitraffic.json"
)

In [ ]:
add_api_tools(
    service_name="marine_digitraffic",
    base_url="https://meri.digitraffic.fi",
    oas_url="https://meri.digitraffic.fi/swagger/openapi.json",
    oas_destination="api/marine_digitraffic.json"
)

In [ ]:
#| export
add_function_tools(
    service_name="ytube_live",
    function_names=[
        "llmcam.fn_to_fc.capture_youtube_live_frame_and_save",
        "llmcam.fn_to_fc.ask_gpt4v_about_image_file"
    ]
)

However, if we have both tools stemming from functions and API requests, we can benefit from an auxiliary function to redirect the calls into sub-auxiliary functions:

In [ ]:
#| export
from llmcam.oas_to_requests import generate_request

def redirect_tool_calls(
    function_name: str,  # Name of the function
    tools: list = [],  # List of tools
    **kwargs  # Keyword arguments
):
    """Redirect tool calls to the appropriate function."""
    for tool in tools:
        if tool["function"]["name"] == function_name:
            if "module" in tool["function"]["parameters"]["properties"]:
                return execute_function(
                    function_name=function_name, 
                    tools=tools, 
                    **kwargs)
            
            elif "url" in tool["function"]["parameters"]["properties"]:
                return generate_request(
                    function_name=function_name, 
                    tools=tools, 
                    **kwargs)
    
    raise ValueError("Function not found")

In [ ]:
show_doc(redirect_tool_calls)

---

### redirect_tool_calls

>      redirect_tool_calls (function_name:str, tools:list=[], **kwargs)

*Redirect tool calls to the appropriate function.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| function_name | str |  | Name of the function |
| tools | list | [] | List of tools |
| kwargs |  |  |  |

Test integrating with a GPT chatbot that has both Road API from DigiTraffic and Youtube Live functions.

In [ ]:
tools = extract_tools_from_services(["ytube_live", "road_digitraffic"])

In [ ]:
#| eval: false
import textwrap
from llmcam.fn_to_fc import complete, form_msg

messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg("user", "Get the weather camera information for the stations with ID C01503 and C01504.")
]
complete(messages, tools, aux_fn=redirect_tool_calls)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Get the weather camera information for the stations with ID C01503 and C01504.
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_3KBOJI5x7LiaAhGl6cW7AyHo', 'function': {'arguments': '{"url": "https://tie.digitraffic.fi/api/weathercam/v1/stations/{id}", "method": "get", "path": {"id": "C01503"}}', 'name': 'weathercamStation'}, 'type': 'function'}, {'id': 'call_9F29M1qJTJRXh4vu6CU0smDx', 'function': {'arguments': '{"url": "https://tie.digitraffic.fi/api/weathercam/v1/stations/{id}", "method": "get", "path": {"id": "C01504"}}', 'name': 'weathercamStation'}, 'type': 'function'}]}
>> Tool:
{"url": "https://tie.digitraffic.fi/api/weathercam/v1/stations/{id}", "method": "get", "path":
{"id": "C01503"}, "weathercamStation": {"type": "Feature", "id": "C01503", "geometry": {"type":
"Point", "coordinates": [23.99616, 60.05374, 0.0]}, "properties": {"

In [ ]:
#| eval: false
messages = [
    form_msg("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    form_msg(
        "user", 
        "Can you capture a YouTube Live and extract information from it? You can use the default link."
    )
]
complete(messages, tools, aux_fn=redirect_tool_calls)
for message in messages:
    print(f">> {message['role'].capitalize()}:")
    try:
        print(textwrap.fill(message["content"], 100))
    except:
        print(message)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information
cap_2024.11.14_00:36:33_unclear.jpg
>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
Can you capture a YouTube Live and extract information from it? You can use the default link.
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_hyZzypALU3DqAm7RttDQSFrr', 'function': {'arguments': '{}', 'name': 'capture_youtube_live_frame_and_save'}, 'type': 'function'}]}
>> Tool:
{"capture_youtube_live_frame_and_save": "../data/cap_2024.11.14_00:36:33_unclear.jpg"}
>> Assistant:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_4JsZGSX3YHh90hyGCvtD

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()